# **Amplitude Envelope (AE)**
The AE is a representation of how the maximum amplitude of a sound varies within specific time frames. It essentially captures the 'shape' of the sound's loudness over time, making it a vital tool for understanding the dynamics and expressiveness of music. By analyzing the AE, we can gain insights into various aspects of a sound, such as its rhythmic structure, intensity changes, and emotional impact.


In [ ]:
%pip install librosa numpy matplotlib

In [ ]:
import librosa
import logging
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import logging

from IPython.display import Audio, display


In [ ]:
# Adjust sys.path to include the root directory
logging.info("Adjusting sys.path to include the root directory")
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
logging.info(f"sys.path adjusted: {sys.path}")

In [ ]:
# Local Imports & Parameters
logging.info("Local configurations and parameters imported")
from config.config import audio_config, output_config
from config.parameters import *
from config.logging import setup_logging

# Plot configurations
from config.matplotlib_plots import configure_plot
logging.info("Local configurations and parameters imported")

In [ ]:
# Set up logging for this notebook
logging.info("Setting up logging for the notebook")
notebook_path = os.path.join(os.getcwd(), 'amplitude_envelopes.ipynb')
setup_logging(notebook_path)
logging.info("Logging set up complete")

In [ ]:
# Set the audio file to analyse
logging.info("Setting the audio file to analyse")
# TODO: change the song to have another analysis
# (pywidget for song upload is not correctly working in Jupyter)
audio_file_key = AUDIO_FILE_SAX_A3
audio_file_path = audio_config.get_audio_file(audio_file_key)

logging.info(f"Audio file set to: {audio_file_path}")

In [ ]:
def analyse_audio(audio_file_path):
    """
    Load an audio file, calculate its amplitude envelope, and plot the original signal
    along with the amplitude envelope.

    Args:
        audio_file_path (str): Path to the audio file.

    Returns:
        tuple: (y, sr, amplitude_envelope, time, t_frames)
            y: Audio time series
            sr: Sampling rate
            amplitude_envelope: Amplitude envelope of the audio
            time: Time vector for the original signal
            t_frames: Time vector for the amplitude envelope
    """
    try:
        logging.info(f"Loading audio file from: {audio_file_path}")
        y, sr = librosa.load(audio_file_path, sr=None)

        frame_size = 2056
        hop_length = 128
        logging.info("Calculating amplitude envelope")
        
        # Vectorized amplitude envelope calculation
        frames = librosa.util.frame(y, frame_length=frame_size, hop_length=hop_length)
        amplitude_envelope = frames.max(axis=0)

        time = np.linspace(0, len(y) / sr, num=len(y))
        frames_count = amplitude_envelope.shape[0]
        t_frames = librosa.frames_to_time(range(frames_count), sr=sr, hop_length=hop_length)

        return y, sr, amplitude_envelope, time, t_frames

    except FileNotFoundError:
        logging.error(f"File not found: {audio_file_path}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

In [ ]:
try:
    y, sr, amplitude_envelope, time, t_frames = analyse_audio(audio_file_path)
    logging.info(f"Successfully analysed audio file {audio_file_path}")

except Exception as e:
    logging.error(f"Error analysing audio file {audio_file_path}: {e}")
    raise

# Display the audio
audio_display = Audio(audio_file_path)
audio_display

In [ ]:
# Function to ensure the directory exists
def ensure_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


In [ ]:
# Plot the original signal and Amplitude Envelope
logging.info("Plotting the original signal and amplitude envelope")
fig, ax = plt.subplots(figsize=(14, 5))

# Extract the file name from the file path
file_name = os.path.basename(audio_file_path)
file_base_name, _ = os.path.splitext(file_name)

# Plot the original signal and amplitude envelope
ax.plot(time, y, color=ORIGINAL_SIGNAL_COLOR, label='Original Signal')
ax.plot(t_frames, amplitude_envelope, color=AMPLITUDE_ENVELOPE_COLOR, label='Amplitude Envelope')

# Configure the plot
configure_plot(ax, title=file_name, subtitle="Original Signal and Amplitude Envelope")

# Ensure the output directory exists
output_directory = output_config.get_output_directory("time_domain")
ensure_directory(output_directory)

# Save the plot
output_path = os.path.join(output_directory, f'AE_{file_base_name}.png')
plt.savefig(output_path, facecolor=BACKGROUND_COLOR)

# Log the saved plot location
logging.info(f"Plot saved to: {output_path}")

# Show the plot
plt.show()